<a href="https://colab.research.google.com/github/ThalyaGIT/UK-Music-Index-Returns/blob/main/3_data-analysis_notebooks/UK_Music_Happiness_and_Index_Returns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Import packages
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate


In [15]:
# Open CSV into dataframe
url_1_day = 'https://raw.githubusercontent.com/ThalyaGIT/UK-Music-Index-Returns/main/0-data-gold/data_1_days.csv'
url_3_day = 'https://raw.githubusercontent.com/ThalyaGIT/UK-Music-Index-Returns/main/0-data-gold/data_3_days.csv'
url_5_day = 'https://raw.githubusercontent.com/ThalyaGIT/UK-Music-Index-Returns/main/0-data-gold/data_5_days.csv'
url_10_day = 'https://raw.githubusercontent.com/ThalyaGIT/UK-Music-Index-Returns/main/0-data-gold/data_10_days.csv'
url_20_day = 'https://raw.githubusercontent.com/ThalyaGIT/UK-Music-Index-Returns/main/0-data-gold/data_20_days.csv'

df_1_day = pd.read_csv(url_1_day)
df_3_day = pd.read_csv(url_3_day)
df_5_day = pd.read_csv(url_5_day)
df_10_day = pd.read_csv(url_10_day)
df_20_day = pd.read_csv(url_20_day)

# # Attempt to convert the 'Date' column to datetime, coercing errors to NaT (Not a Time)
df_1_day['Date'] = pd.to_datetime(df_1_day['Date'], errors='coerce')



# **Next Period**

In [16]:
import pandas as pd
from tabulate import tabulate

# Initialize an empty list to store results
results = []

indices = ['GILT2', 'FTSE100', 'MSCIUK', 'FTSEAllShare', 'FTSE250', 'FTSESmallCap','FTSEAIM']
days_list = [1, 3, 5, 10, 20]

for days in days_list:
    result_row = [days]
    for index in indices:
        df = globals()[f'df_{days}_day']  # Dynamically access each DataFrame

        # Ensure 'Date' column is in datetime format
        df['Date'] = pd.to_datetime(df['Date'])

        y = df[f'Next % {index} Change']
        X = df[['Change in SWAV',
                'ADS_Change',
                'EPU_Change',
                f'% {index} Change',
                'Next % MSCI Change',
                'Vix Close',
                'Rolling_Avg_Change_in_DCC',
                'Stringency_Change']]

        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()

        coef = round(model.params['Change in SWAV'], 2)
        p_value = round(model.pvalues['Change in SWAV'], 5)

        result_row.extend([coef, p_value])  # Make sure you add both coef and p-value

    # Append the result row for this combination of days
    results.append(result_row)

# Define column names dynamically
columns = ['Days']
for index in indices:
    columns.extend([f'{index} Coef', f'{index} P-value'])  # Ensure you have columns for both coef and p-value

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results, columns=columns)

# Define a function to apply the styling
def color_rows(row):
    colors = []
    for i in range(1, len(row), 2):  # Skip Days, then iterate through Coefs and P-values
        coef = row[i]
        p_value = row[i+1]
        if p_value < 0.1:  # Consider p-values < 0.1 as significant
            if coef > 0:
                colors.extend(['background-color: green', ''])
            elif coef < 0:
                colors.extend(['background-color: red', ''])
        else:
            colors.extend(['background-color: #d3d3d3', ''])  # Use a lighter grey color
    return [''] * 1 + colors  # No coloring for Days

# Apply the function to each row of the DataFrame
styled_df = results_df.style.apply(color_rows, axis=1)

# Display the styled DataFrame
styled_df

<ipython-input-16-785942667c55>:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coef = row[i]
<ipython-input-16-785942667c55>:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = row[i+1]


,Days,GILT2 Coef,GILT2 P-value,FTSE100 Coef,FTSE100 P-value,MSCIUK Coef,MSCIUK P-value,FTSEAllShare Coef,FTSEAllShare P-value,FTSE250 Coef,FTSE250 P-value,FTSESmallCap Coef,FTSESmallCap P-value,FTSEAIM Coef,FTSEAIM P-value
0,1,-288.330000,0.350050,-1.340000,0.369000,-0.090000,0.952300,-1.450000,0.304930,-2.250000,0.164060,-0.780000,0.546380,-2.120000,0.118000
1,3,-23.870000,0.865640,-0.320000,0.858680,-1.850000,0.293620,-0.250000,0.881840,-0.600000,0.762500,-0.020000,0.987990,-2.590000,0.157820
2,5,319.550000,0.535170,-0.640000,0.733570,-3.070000,0.093280,-0.210000,0.904630,1.790000,0.365410,0.970000,0.547960,1.170000,0.517430
3,10,330.660000,0.118600,-1.930000,0.283550,-6.870000,0.000230,-1.560000,0.351300,0.260000,0.885080,1.180000,0.420300,0.990000,0.550820
4,20,-826.290000,0.200900,1.250000,0.422100,-5.640000,0.001400,0.480000,0.746830,-2.830000,0.081260,0.140000,0.912880,-1.720000,0.235760


In [18]:
# import pandas as pd
# from tabulate import tabulate

# # Initialize an empty list to store results
# results = []

# indices = ['GILT2', 'FTSE100', 'MSCIUK', 'FTSEAllShare', 'FTSE250', 'FTSESmallCap','FTSEAIM']
# days_list = [1, 3, 5, 10, 20]

# for days in days_list:
#     result_row = [days]
#     for index in indices:
#         df = globals()[f'df_{days}_day']  # Dynamically access each DataFrame

#         # Ensure 'Date' column is in datetime format
#         df['Date'] = pd.to_datetime(df['Date'])

#         y = df[f'% {index} Change']
#         X = df[['Change in SWAV',
#                 'ADS_Change',
#                 'EPU_Change',
#                 f'Previous % {index} Change',
#                 '% MSCI Change',
#                 'Vix Close',
#                 'Rolling_Avg_Change_in_DCC',
#                 'Stringency_Change']]

#         X = sm.add_constant(X)
#         model = sm.OLS(y, X).fit()

#         coef = round(model.params['Change in SWAV'], 2)
#         p_value = round(model.pvalues['Change in SWAV'], 5)

#         result_row.extend([coef, p_value])  # Make sure you add both coef and p-value

#     # Append the result row for this combination of days
#     results.append(result_row)

# # Define column names dynamically
# columns = ['Days']
# for index in indices:
#     columns.extend([f'{index} Coef', f'{index} P-value'])  # Ensure you have columns for both coef and p-value

# # Convert the results list to a DataFrame
# results_df = pd.DataFrame(results, columns=columns)

# # Define a function to apply the styling
# def color_rows(row):
#     colors = []
#     for i in range(1, len(row), 2):  # Skip Days, then iterate through Coefs and P-values
#         coef = row[i]
#         p_value = row[i+1]
#         if p_value < 0.1:  # Consider p-values < 0.1 as significant
#             if coef > 0:
#                 colors.extend(['background-color: green', ''])
#             elif coef < 0:
#                 colors.extend(['background-color: red', ''])
#         else:
#             colors.extend(['background-color: #d3d3d3', ''])  # Use a lighter grey color
#     return [''] * 1 + colors  # No coloring for Days

# # Apply the function to each row of the DataFrame
# styled_df = results_df.style.apply(color_rows, axis=1)

# # Display the styled DataFrame
# styled_df

<ipython-input-18-0bdf013e3ca4>:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coef = row[i]
<ipython-input-18-0bdf013e3ca4>:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = row[i+1]


,Days,GILT2 Coef,GILT2 P-value,FTSE100 Coef,FTSE100 P-value,MSCIUK Coef,MSCIUK P-value,FTSEAllShare Coef,FTSEAllShare P-value,FTSE250 Coef,FTSE250 P-value,FTSESmallCap Coef,FTSESmallCap P-value,FTSEAIM Coef,FTSEAIM P-value
0,1,-12.680000,0.927810,-0.980000,0.510320,0.310000,0.814380,-0.740000,0.602990,0.310000,0.848540,0.970000,0.443960,1.870000,0.151660
1,3,78.330000,0.806960,-4.210000,0.020900,-2.100000,0.232310,-3.430000,0.044790,-0.370000,0.854040,1.480000,0.367650,0.230000,0.900010
2,5,-37.310000,0.941320,-5.100000,0.007260,-4.200000,0.025630,-4.290000,0.015450,-1.130000,0.572410,2.120000,0.223320,-1.400000,0.474820
3,10,-92.790000,0.819630,-6.510000,0.001060,-6.690000,0.001210,-5.780000,0.001690,-3.090000,0.118940,0.370000,0.840520,-7.200000,0.001100
4,20,-1052.390000,0.065220,-5.350000,0.008170,-7.520000,0.000980,-5.220000,0.005350,-4.470000,0.023720,-0.490000,0.802340,-11.090000,0.000000


In [17]:
# import pandas as pd
# from tabulate import tabulate

# # Initialize an empty list to store results
# results = []

# indices = ['GILT2', 'FTSE100', 'MSCIUK', 'FTSEAllShare', 'FTSE250', 'FTSESmallCap','FTSEAIM']
# days_list = [1, 3, 5, 10, 20]
# years_list = range(2017, 2023)  # Years from 2017 to 2022

# for year in years_list:
#     for days in days_list:
#         result_row = [year, days]
#         for index in indices:
#             df = globals()[f'df_{days}_day']  # Dynamically access each DataFrame

#             # Ensure 'Date' column is in datetime format
#             df['Date'] = pd.to_datetime(df['Date'])
#             # Filter for the specific year
#             df_filtered = df[df['Date'].dt.year == year]

#             y = df_filtered[f'% {index} Change']
#             X = df_filtered[['Change in SWAV',
#                              'ADS_Change',
#                              'EPU_Change',
#                              f'Previous % {index} Change',
#                              '% MSCI Change',
#                              'Vix Close',
#                              'Rolling_Avg_Change_in_DCC',
#                              'Stringency_Change']]

#             X = sm.add_constant(X)
#             model = sm.OLS(y, X).fit()

#             coef = round(model.params['Change in SWAV'], 2)
#             p_value = round(model.pvalues['Change in SWAV'], 5)

#             result_row.extend([coef, p_value])

#         # Append the result row for this combination of year and days
#         results.append(result_row)

# # Define column names dynamically
# columns = ['Year', 'Days']
# for index in indices:
#     columns.extend([f'{index} Coef', f'{index} P-value'])

# # Convert the results list to a DataFrame
# results_df = pd.DataFrame(results, columns=columns)

# # Define a function to apply the styling
# def color_rows(row):
#     colors = []
#     for i in range(2, len(row), 2):  # Skip Year and Days, then iterate through Coefs and P-values
#         coef = row[i]
#         p_value = row[i+1]
#         if p_value < 0.05:  # Consider p-values < 0.05 as significant
#             if coef > 0:
#                 colors.extend(['background-color: green', ''])
#             elif coef < 0:
#                 colors.extend(['background-color: red', ''])
#         else:
#             colors.extend(['background-color: grey', ''])
#     return [''] * 2 + colors  # No coloring for Year and Days

# # Apply the function to each row of the DataFrame
# styled_df = results_df.style.apply(color_rows, axis=1)

# # Display the styled DataFrame
# styled_df

<ipython-input-17-608d5c78b8fa>:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coef = row[i]
<ipython-input-17-608d5c78b8fa>:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = row[i+1]


,Year,Days,GILT2 Coef,GILT2 P-value,FTSE100 Coef,FTSE100 P-value,MSCIUK Coef,MSCIUK P-value,FTSEAllShare Coef,FTSEAllShare P-value,FTSE250 Coef,FTSE250 P-value,FTSESmallCap Coef,FTSESmallCap P-value,FTSEAIM Coef,FTSEAIM P-value
0,2017,1,224.810000,0.045930,1.040000,0.722270,1.360000,0.628750,0.570000,0.830290,-1.550000,0.561100,-0.100000,0.960830,1.920000,0.386910
1,2017,3,-8.890000,0.928780,-1.050000,0.772090,-4.510000,0.178180,-1.110000,0.737200,-1.560000,0.627860,-1.650000,0.491120,1.570000,0.614990
2,2017,5,-293.020000,0.044240,-0.000000,0.999260,-9.390000,0.028290,0.190000,0.963320,1.220000,0.776660,0.120000,0.970270,0.280000,0.948990
3,2017,10,-152.090000,0.352600,6.660000,0.167070,-0.130000,0.977260,5.830000,0.190830,4.370000,0.355590,0.310000,0.927750,-0.800000,0.873970
4,2017,20,-208.860000,0.285110,2.760000,0.582190,19.670000,0.000260,5.610000,0.222680,16.010000,0.001490,7.120000,0.065010,12.320000,0.043180
5,2018,1,-7.940000,0.658880,6.250000,0.072350,3.470000,0.223810,6.250000,0.054940,6.810000,0.034720,4.190000,0.081890,6.660000,0.027680
6,2018,3,31.870000,0.174870,6.020000,0.170920,7.300000,0.070630,6.150000,0.128690,7.450000,0.064160,3.470000,0.194530,4.230000,0.321830
7,2018,5,-1.620000,0.952230,-1.820000,0.706500,-6.220000,0.147820,-1.300000,0.769630,0.410000,0.926100,-0.390000,0.896700,-6.590000,0.184970
8,2018,10,-17.680000,0.571270,-1.220000,0.844230,-5.090000,0.334860,-2.340000,0.682860,-8.340000,0.122700,-2.810000,0.464390,-18.030000,0.002950
9,2018,20,-101.770000,0.002090,16.990000,0.026880,-2.570000,0.711230,15.840000,0.027050,10.990000,0.098830,8.440000,0.087450,-7.230000,0.311250
